# Tutorial 

## Imports

In [2]:
from formula import *
from quantifier import *
from exh import *

## Create formulas

Formulas are created from propositional variables. Variables have indices, such that variables with the same index always have the same truth-value and variables with different indices are logically independent. Here is a code to create a variable with index 4.

In [3]:
d = Var(4, name = "d") 
d1 = Var(7) # "name" is optional, "name" makes prettier display with print and helps for evaluation of formulas

By default, *formula.py* creates 3 variables a,b and c with indices 0, 1, 2 respectively. Once some variables are defined, one can create complex formulas with & (and), | (or) and \~ (not).

In [4]:
f1 = a | b
print(f1)
f2 = a & b
print(f2)
f3 = a | (~a & b)
print(f3)

a or b
a and b
a or (not[a] and b)


To turn a propositional variable into a predicate (for use in quantified formulas), one needs to specify the names of the variables it depends on.

In [5]:
d.depends_on("x")

Once this is done, one can use the following syntax to create a universal statement (using module *quantifier.py*).  A("x") creates a quantifier over "x", which combines with ">" and a formula to form a quantified formula.

In [6]:
f4 = A("x") > d
print(f4)

∀x, d(x)


Two simplifying tips:
    - Existential and universal quantifiers over x, y, z are created by default under the names Ax, Ey, etc.
    - If v is propositional variable, v("x") returns v and sets v to depend on x
Most straigthforwardly, one can create a quantified formula as follows

In [7]:
f5 = Ey > d1("y")
print(f5) # displays as A7 because we haven't given d1 a name

∃y, A7(y)


## Evaluate formulas
The simplest way to evaluate a formula uses the method *evaluate* and the variables' names.

In [8]:
print("'{}'".format(f1), "is", f1.evaluate(a = True, b = False))

'a or b' is True


In quantified formulas, one needs to provide as many values for a predicate as there are individuals in the domain. The number of individuals in the domain is set in *options.py* and defaults to 3. The three values are provided as a list.

In [9]:
print("'{}'".format(f4), "is", f4.evaluate(d = [True, True, True]))

'∀x, d(x)' is True


One may need to be more systematic and evaluate the formula over a larget set of assignments of values. The module *universe.py* allows to create the set of worlds with all possible combinations of values for predicates. The *truth_table* method displays the evaluated formula in a fancy chart.

In [10]:
from worlds import Universe

# A Universe can be created from formulas ; the cosntructor extracts all the independent predicates and propositional variables
prop_universe = Universe(fs = [a, b, c])

print(prop_universe.evaluate(f1, f2, f3))
prop_universe.truth_table(f1, f2, f3)


[[False False False]
 [ True False  True]
 [ True False  True]
 [ True  True  True]
 [False False False]
 [ True False  True]
 [ True False  True]
 [ True  True  True]]
3


a,b,c,a or b,a and b,a or (not[a] and b)
False,False,False,False,False,False
True,False,False,True,False,True
False,True,False,True,False,True
True,True,False,True,True,True
False,False,True,False,False,False
True,False,True,True,False,True
False,True,True,True,False,True
True,True,True,True,True,True


Universes come with methods to check standard logical relations: entailment, equivalence and consistency

In [11]:
# The first three propositions entail that "not a" and "b", contradicting the fourth. 
print(prop_universe.consistent(
                                a | b,
                               ~a | ~b,
                               b | ~a,
                               a | ~b 
                              )) 
print(prop_universe.entails(
                            a | ~ b & c,
                            ~(b & ~a) 
                           )) 
print(prop_universe.equivalent(
                                ~a | ~c,
                               ~(a & b) 
                              )) # De Morgan's law

False
True
False


## Exhaustification

Exhaustification can be computed against a set of stipulated alternatives.

**NB:** Innocent exclusions is computed upon creation of the object, expect slowdowns if number of worlds is big.

In [12]:
e = Exh(Ex > d, alts = [Ax > d])

There are a couple of ways to know what has been computed. First, the method *diagnose* gives us the innocently excludable alternatives. Second, like any formula, we can evaluate *e* and check that it behaves like "some but not all". Third, we can create a *Universe* object and check for equivalence with a known formula

In [13]:
e.diagnose()

print()
print("When all elements are True of the predicate, e is", e.evaluate(d = [True, True, True]))
print("When some but not all are, e is", e.evaluate(d = [True, False, True]))
print("When none is, e is", e.evaluate(d = [False, False, False]))

quant_universe = Universe(fs = [e])
print()
print("Is e equivalent to 'some but not all'?")
print(
    quant_universe.equivalent(
    e,
    (Ex > d) & ~(Ax > d)
    ))



Maximal Sets (excl)
[∀x, d(x)]

Innocently excludable ∀x, d(x)


When all elements are True of the predicate, e is False
When some but not all are, e is True
When none is, e is False

Is e equivalent to 'some but not all'?
True


Below is a more involved example with more alternatives:

In [14]:
p1 = Var(5, name = "p1")("x") # constructing a new predicate and immediately indicating dependency in x
p2 = Var(6, name = "p2")("x")

prejacent = Ax > p1 | p2

exh = Exh(prejacent, alts = [Ax > p1 & p2,
                             Ax > p1,
                             Ax > p2,
                             Ex > p1 & p2,
                             Ex > p1,
                             Ex > p2])
exh.diagnose()
# Reads like "none of them did both p1 and p2" ; an embedded implicature
# What if we didn't have existential alternatives?

exh2 = Exh(prejacent, alts = [Ax > p1 & p2,
                              Ax > p1,
                              Ax > p2])
exh2.diagnose()
universe = Universe(fs = [exh2])
print(universe.entails(exh2, Ex > p1 & ~p2))
print(universe.entails(exh2, Ex > p2 & ~p1))
# Two implicatures: 1) that someone did only p1, 2) that someone did only p2

Maximal Sets (excl)
[∀x, p1(x) and p2(x), ∀x, p2(x), ∃x, p1(x) and p2(x), ∃x, p2(x)]
[∀x, p1(x) and p2(x), ∀x, p1(x), ∀x, p2(x), ∃x, p1(x) and p2(x)]
[∀x, p1(x) and p2(x), ∀x, p1(x), ∃x, p1(x) and p2(x), ∃x, p1(x)]

Innocently excludable ∀x, p1(x) and p2(x); ∃x, p1(x) and p2(x)

Maximal Sets (excl)
[∀x, p1(x) and p2(x), ∀x, p1(x), ∀x, p2(x)]

Innocently excludable ∀x, p1(x) and p2(x); ∀x, p1(x); ∀x, p2(x)

True
True


## Advanced usage

### Formulas with multiple quantifiers

One can create multiply quantified sentences ; the number of worlds grows exponentially. One predicate that depends on two variables will give rise to 9 independent variables ; we get 2^9 = 512 worlds.

In [15]:
p3 = Var(13, name = "d3")
prejacent = Ex > Ay > p3("x", "y")

e = Exh(prejacent, alts = [Ay > Ex > p3, Ax > Ey > p3, Ey > Ax > p3])
e.diagnose()

Maximal Sets (excl)
[∀x, ∃y, d3(x,y), ∃y, ∀x, d3(x,y)]

Innocently excludable ∀x, ∃y, d3(x,y); ∃y, ∀x, d3(x,y)



### Constrained universe